In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#%cd /content/drive/MyDrive/FINAL DATA

/content/drive/MyDrive/FINAL DATA


Importing libraries

In [ ]:
import pickle
import numpy as np
import os
import random

import sklearn

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import svm #support vector machine
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC #support vector classifier
from sklearn.metrics import accuracy_score
from tabulate import tabulate

Getting feature's names with col indexing

In [ ]:
'''features = pickle.load(open("colIndDict.pkl", "rb"))
f_list=list(features)
print(features)
print("\n\n")
print(f_list)'''

{'binary': [0, 163], 'CKSNAP1': [164, 195], 'CKSNAP3': [196, 227], 'CKSNAP5': [228, 259], 'CKSNAP7': [260, 291], 'CKSNAP9': [292, 323], 'DAC7': [324, 365], 'EIIP': [366, 406], 'kmer1': [407, 410], 'kmer2': [411, 430], 'kmer3': [431, 514], 'kmer4': [515, 854], 'kmer5': [855, 2218], 'NCP': [2219, 2341], 'PseEIIP': [2342, 2405], 'slwindow10': [2406, 2553], 'slwindow5': [2554, 2701], 'TAC7': [2702, 2715]}



['binary', 'CKSNAP1', 'CKSNAP3', 'CKSNAP5', 'CKSNAP7', 'CKSNAP9', 'DAC7', 'EIIP', 'kmer1', 'kmer2', 'kmer3', 'kmer4', 'kmer5', 'NCP', 'PseEIIP', 'slwindow10', 'slwindow5', 'TAC7']


I did some mistakes in my y_train and y_test data

In [ ]:
x_train = np.load('x_train.npy')
#y_train = np.load('y_train.npy')
x_test = np.load('x_test.npy')
#y_test = np.load('y_test.npy')

In [ ]:
###########y_test########

temp1 = []
for i in range(262):
    if i<=130:
        temp1.append(0)
    else:
        temp1.append(1)

#print(temp1)

arr = np.array(temp1)
# Convert 1D array to a 2D numpy array of 262 rows and 1 columns
y_test = np.reshape(arr, (262, 1))
print(y_test.shape)

(262, 1)


In [ ]:
###########y_train########

temp2 = []
for i in range(3690):
    if i<1845:
        temp2.append(0)
    else:
        temp2.append(1)

#print(temp2)

arr = np.array(temp2)
# Convert 1D array to a 2D numpy array of 3690 rows and 1 columns
y_train = np.reshape(arr, (3690, 1))
print(y_train)
print(y_train.shape)

[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
(3690, 1)


Importing train & test data

Deviding the 18 features into 5 groups with 5 unique random features in each

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
#print(y_test)


dummy_features = {1 : 'binary', 2 : 'CKSNAP1', 3 : 'CKSNAP3', 4 : 'CKSNAP5', 5 : 'CKSNAP7', 6 : 'CKSNAP9', 7 : 'DAC7', 8 : 'EIIP', 9 : 'kmer1', 10 : 'kmer2', 11 : 'kmer3', 12 : 'kmer4', 13 : 'kmer5', 14 : 'NCP', 15 : 'PseEIIP', 16 : 'slwindow10', 17 : 'slwindow5', 18 : 'TAC7'}

temp = random.sample(range(1,18),5)
subgrp1 = [dummy_features[i] for i in (temp)]

temp = random.sample(range(1,18),5)
subgrp2 = [dummy_features[i] for i in (temp)]

temp = random.sample(range(1,18),5)
subgrp3 = [dummy_features[i] for i in (temp)]

temp = random.sample(range(1,18),5)
subgrp4 = [dummy_features[i] for i in (temp)]

temp = random.sample(range(1,18),5)
subgrp5 = [dummy_features[i] for i in (temp)]

print(subgrp1)
print(subgrp2)
print(subgrp3)
print(subgrp4)
print(subgrp5)

(3690, 2716)
(3690, 1)
(262, 2716)
(262, 1)
['slwindow5', 'PseEIIP', 'CKSNAP5', 'kmer1', 'DAC7']
['CKSNAP9', 'DAC7', 'kmer1', 'NCP', 'kmer5']
['PseEIIP', 'CKSNAP1', 'kmer1', 'NCP', 'kmer4']
['DAC7', 'CKSNAP3', 'kmer5', 'kmer2', 'PseEIIP']
['NCP', 'slwindow5', 'DAC7', 'kmer3', 'EIIP']


Concatenating the data group wise

In [ ]:
x1_train = np.concatenate([x_train[ : ,  features[x][0] : features[x][1]+1] for x in subgrp1], axis = 1)
x1_test = np.concatenate([x_test[ : ,  features[x][0] : features[x][1]+1] for x in subgrp1], axis = 1)

x2_train = np.concatenate([x_train[ : ,  features[x][0] : features[x][1]+1] for x in subgrp2], axis = 1)
x2_test = np.concatenate([x_test[ : ,  features[x][0] : features[x][1]+1] for x in subgrp2], axis = 1)

x3_train = np.concatenate([x_train[ : ,  features[x][0] : features[x][1]+1] for x in subgrp3], axis = 1)
x3_test = np.concatenate([x_test[ : ,  features[x][0] : features[x][1]+1] for x in subgrp3], axis = 1)

x4_train = np.concatenate([x_train[ : ,  features[x][0] : features[x][1]+1] for x in subgrp4], axis = 1)
x4_test = np.concatenate([x_test[ : ,  features[x][0] : features[x][1]+1] for x in subgrp4], axis = 1)

x5_train = np.concatenate([x_train[ : ,  features[x][0] : features[x][1]+1] for x in subgrp5], axis = 1)
x5_test = np.concatenate([x_test[ : ,  features[x][0] : features[x][1]+1] for x in subgrp5], axis = 1)

print(x1_train.shape , x2_train.shape, x3_train.shape, x4_train.shape, x5_train.shape)
print(x1_test.shape , x2_test.shape, x3_test.shape, x4_test.shape, x5_test.shape)

x_train_set = [x1_train, x2_train, x3_train, x4_train, x5_train]
x_test_set = [x1_test, x2_test, x3_test, x4_test, x5_test]

#print(x_train_set[0].shape)

(3690, 290) (3690, 1565) (3690, 563) (3690, 1522) (3690, 438)
(262, 290) (262, 1565) (262, 563) (262, 1522) (262, 438)


In [ ]:
#cross_validation = StratifiedKFold(n_splits=5, shuffle=True)
kf = KFold(n_splits=5, shuffle=True)

In [ ]:
#defining classifiers
svc = SVC()
logistic_regression = LogisticRegression(max_iter=500) #default a error dicche
xgb_classifier = XGBClassifier()
adaboost_classifier = AdaBoostClassifier()
random_forest_classifier = RandomForestClassifier()

Defining some list to keep accuracy, mcc, sens, spes values

In [ ]:
train_accuracy = []
test_accuracy = []
independent_accuracy = []

train_mcc = [] #The Matthews correlation coefficient (MCC)
test_mcc = []
independent_mcc = []

train_sensitivity = []
test_sensitivity = []
independent_sensitivity = []

train_specificity = []
test_specificity = []
independent_specificity = []

In [ ]:
for k in range(5):  
    print(x_test_set[k].shape)

(262, 290)
(262, 1565)
(262, 563)
(262, 1522)
(262, 438)


In [ ]:
############FOR SVC###########

j=-1
for x in (x_train_set):
    j = j+1
    #print(x.shape)
    #print(y_train.shape)
    for train_index, test_index in kf.split(x, y_train):
        subgrp_x_train = x[train_index]
        #print(train_index)
        subgrp_x_test = x[test_index]
        #print(train_index)

        subgrp_y_train = y_train[train_index]
        subgrp_y_test = y_train[test_index]
        #print(subgrp_x_train.shape)
        #print(subgrp_x_test.shape)

        svc.fit(subgrp_x_train, subgrp_y_train)#machine k train korlam
        train_pred = svc.predict(subgrp_x_train)#
        test_pred = svc.predict(subgrp_x_test)
        independent_pred = svc.predict(x_test_set[j])


        #print(type(train_pred), type(test_pred), type(independent_pred))
        #print(train_pred.shape, test_pred.shape, independent_pred.shape)

        
        
        ###############ACCURACY#################
        train_accuracy.append(accuracy_score(subgrp_y_train, train_pred))
        test_accuracy.append(accuracy_score(subgrp_y_test, test_pred))
        independent_accuracy.append(accuracy_score(y_test, independent_pred))


        ###############MCC#################
        train_mcc.append(matthews_corrcoef(subgrp_y_train, train_pred))
        test_mcc.append(matthews_corrcoef(subgrp_y_test, test_pred))
        independent_mcc.append(matthews_corrcoef(y_test, independent_pred))


        ###############Sensitivity & Specificity#################
        tn, fp, fn, tp = confusion_matrix(subgrp_y_train, train_pred).ravel()
        train_sensitivity.append(tn/(tn+fp))
        train_specificity.append(tp/(tp+fn))

        tn, fp, fn, tp = confusion_matrix(subgrp_y_test, test_pred).ravel()
        test_sensitivity.append(tn/(tn+fp))
        test_specificity.append(tp/(tp+fn))

        tn, fp, fn, tp = confusion_matrix(y_test, independent_pred).ravel()
        independent_sensitivity.append(tn/(tn+fp))
        independent_specificity.append(tp/(tp+fn))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [ ]:
for k in range (25):
    print("Index:", k)
    print("Train_accuracy:", train_accuracy[k])
    print("Test_accuracy:", test_accuracy[k])
    print("Independent_accuracy", independent_accuracy[k], "\n\n")

Index: 0
Train_accuracy: 1.0
Test_accuracy: 1.0
Independent_accuracy 0.9847328244274809 


Index: 1
Train_accuracy: 1.0
Test_accuracy: 1.0
Independent_accuracy 0.9885496183206107 


Index: 2
Train_accuracy: 1.0
Test_accuracy: 1.0
Independent_accuracy 0.9885496183206107 


Index: 3
Train_accuracy: 1.0
Test_accuracy: 1.0
Independent_accuracy 0.9885496183206107 


Index: 4
Train_accuracy: 1.0
Test_accuracy: 1.0
Independent_accuracy 0.9885496183206107 


Index: 5
Train_accuracy: 0.8556910569105691
Test_accuracy: 0.7533875338753387
Independent_accuracy 0.8053435114503816 


Index: 6
Train_accuracy: 0.8638211382113821
Test_accuracy: 0.7655826558265583
Independent_accuracy 0.7824427480916031 


Index: 7
Train_accuracy: 0.8553523035230353
Test_accuracy: 0.7886178861788617
Independent_accuracy 0.8053435114503816 


Index: 8
Train_accuracy: 0.8563685636856369
Test_accuracy: 0.7506775067750677
Independent_accuracy 0.8015267175572519 


Index: 9
Train_accuracy: 0.8590785907859079
Test_accuracy: 0.

In [ ]:
print("#####SVC-SUMARRY#####")
print("\n")

table_data = ['SVC']

print("Average Cross-Validated Training Accuracy:", sum(train_accuracy)/len(train_accuracy))
table_data.append(sum(train_accuracy)/len(train_accuracy))
print("Average Cross-Validated Testing Accuracy:", sum(test_accuracy)/len(test_accuracy))
table_data.append(sum(test_accuracy)/len(test_accuracy))
print("Average Independent Accuracy:", sum(independent_accuracy)/len(independent_accuracy))
table_data.append(sum(independent_accuracy)/len(independent_accuracy))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Matthews correlation coefficient:", sum(train_mcc)/len(train_mcc))
table_data.append(sum(train_mcc)/len(train_mcc))
print("Average Cross-Validated Testing Matthews correlation coefficient:", sum(test_mcc)/len(test_mcc))
table_data.append(sum(test_mcc)/len(test_mcc))
print("Average Independent Matthews correlation coefficient:", sum(independent_mcc)/len(independent_mcc))
table_data.append(sum(independent_mcc)/len(independent_mcc))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Sensitivity:", sum(train_sensitivity)/len(train_sensitivity))
table_data.append(sum(train_sensitivity)/len(train_sensitivity))
print("Average Cross-Validated Testing Sensitivity:", sum(test_sensitivity)/len(test_sensitivity))
table_data.append(sum(test_sensitivity)/len(test_sensitivity))
print("Average Independent Sensitivity:", sum(independent_sensitivity)/len(independent_sensitivity))
table_data.append(sum(independent_sensitivity)/len(independent_sensitivity))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Specificity:", sum(train_specificity)/len(train_specificity))
table_data.append(sum(train_specificity)/len(train_specificity))
print("Average Cross-Validated Testing Specificity:", sum(test_specificity)/len(test_specificity))
table_data.append(sum(test_specificity)/len(test_specificity))
print("Average Independent Specificity:", sum(independent_specificity)/len(independent_specificity))
table_data.append(sum(independent_specificity)/len(independent_specificity))
print("\n--------------------------------------------------\n")

print(table_data)

#####SVC-SUMARRY#####


Average Cross-Validated Training Accuracy: 0.8994850948509484
Average Cross-Validated Testing Accuracy: 0.8400542005420053
Average Independent Accuracy: 0.8429007633587788

--------------------------------------------------

Average Cross-Validated Training Matthews correlation coefficient: 0.7990628026593263
Average Cross-Validated Testing Matthews correlation coefficient: 0.6805106884266681
Average Independent Matthews correlation coefficient: 0.6871163487072022

--------------------------------------------------

Average Cross-Validated Training Sensitivity: 0.9049168398586963
Average Cross-Validated Testing Sensitivity: 0.8464261884374727
Average Independent Sensitivity: 0.8607633587786258

--------------------------------------------------

Average Cross-Validated Training Specificity: 0.8940244927103422
Average Cross-Validated Testing Specificity: 0.8339794375571788
Average Independent Specificity: 0.8250381679389314

--------------------------------------

In [ ]:
############FOR LOGISTIC REGRESSION###########

train_accuracy.clear()
test_accuracy.clear()
independent_accuracy.clear()

train_mcc.clear() #The Matthews correlation coefficient (MCC)
test_mcc.clear()
independent_mcc.clear()

train_sensitivity.clear()
test_sensitivity.clear()
independent_sensitivity.clear()

train_specificity.clear()
test_specificity.clear()
independent_specificity.clear()

j=-1
for x in (x_train_set):
    j = j+1
    #print(x.shape)
    #print(y_train.shape)
    for train_index, test_index in kf.split(x, y_train):
        subgrp_x_train = x[train_index]
        subgrp_x_test = x[test_index]

        subgrp_y_train = y_train[train_index]
        subgrp_y_test = y_train[test_index]
        #print(subgrp_x_train.shape)
        #print(subgrp_x_test.shape)

        logistic_regression.fit(subgrp_x_train, subgrp_y_train)
        train_pred = logistic_regression.predict(subgrp_x_train)
        test_pred = logistic_regression.predict(subgrp_x_test)
        independent_pred = logistic_regression.predict(x_test_set[j])


        #print(type(train_pred), type(test_pred), type(independent_pred))
        #print(train_pred.shape, test_pred.shape, independent_pred.shape)

        
        
        ###############ACCURACY#################
        train_accuracy.append(accuracy_score(subgrp_y_train, train_pred))
        test_accuracy.append(accuracy_score(subgrp_y_test, test_pred))
        independent_accuracy.append(accuracy_score(y_test, independent_pred))


        ###############MCC#################
        train_mcc.append(matthews_corrcoef(subgrp_y_train, train_pred))
        test_mcc.append(matthews_corrcoef(subgrp_y_test, test_pred))
        independent_mcc.append(matthews_corrcoef(y_test, independent_pred))


        ###############Sensitivity & Specificity#################
        tn, fp, fn, tp = confusion_matrix(subgrp_y_train, train_pred).ravel()
        train_sensitivity.append(tn/(tn+fp))
        train_specificity.append(tp/(tp+fn))

        tn, fp, fn, tp = confusion_matrix(subgrp_y_test, test_pred).ravel()
        test_sensitivity.append(tn/(tn+fp))
        test_specificity.append(tp/(tp+fn))

        tn, fp, fn, tp = confusion_matrix(y_test, independent_pred).ravel()
        independent_sensitivity.append(tn/(tn+fp))
        independent_specificity.append(tp/(tp+fn))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [ ]:
print("#####LOGISTIC REGRESSION-SUMARRY#####")
print("\n")

table_data.append('LRC')

print("Average Cross-Validated Training Accuracy:", sum(train_accuracy)/len(train_accuracy))
table_data.append(sum(train_accuracy)/len(train_accuracy))
print("Average Cross-Validated Testing Accuracy:", sum(test_accuracy)/len(test_accuracy))
table_data.append(sum(test_accuracy)/len(test_accuracy))
print("Average Independent Accuracy:", sum(independent_accuracy)/len(independent_accuracy))
table_data.append(sum(independent_accuracy)/len(independent_accuracy))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Matthews correlation coefficient:", sum(train_mcc)/len(train_mcc))
table_data.append(sum(train_mcc)/len(train_mcc))
print("Average Cross-Validated Testing Matthews correlation coefficient:", sum(test_mcc)/len(test_mcc))
table_data.append(sum(test_mcc)/len(test_mcc))
print("Average Independent Matthews correlation coefficient:", sum(independent_mcc)/len(independent_mcc))
table_data.append(sum(independent_mcc)/len(independent_mcc))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Sensitivity:", sum(train_sensitivity)/len(train_sensitivity))
table_data.append(sum(train_sensitivity)/len(train_sensitivity))
print("Average Cross-Validated Testing Sensitivity:", sum(test_sensitivity)/len(test_sensitivity))
table_data.append(sum(test_sensitivity)/len(test_sensitivity))
print("Average Independent Sensitivity:", sum(independent_sensitivity)/len(independent_sensitivity))
table_data.append(sum(independent_sensitivity)/len(independent_sensitivity))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Specificity:", sum(train_specificity)/len(train_specificity))
table_data.append(sum(train_specificity)/len(train_specificity))
print("Average Cross-Validated Testing Specificity:", sum(test_specificity)/len(test_specificity))
table_data.append(sum(test_specificity)/len(test_specificity))
print("Average Independent Specificity:", sum(independent_specificity)/len(independent_specificity))
table_data.append(sum(independent_specificity)/len(independent_specificity))
print("\n--------------------------------------------------\n")

print(table_data)

#####LOGISTIC REGRESSION-SUMARRY#####


Average Cross-Validated Training Accuracy: 0.8645257452574526
Average Cross-Validated Testing Accuracy: 0.8462872628726288
Average Independent Accuracy: 0.8593893129770993

--------------------------------------------------

Average Cross-Validated Training Matthews correlation coefficient: 0.7291656565288082
Average Cross-Validated Testing Matthews correlation coefficient: 0.6930739926913083
Average Independent Matthews correlation coefficient: 0.7200917908024406

--------------------------------------------------

Average Cross-Validated Training Sensitivity: 0.8689145881966401
Average Cross-Validated Testing Sensitivity: 0.852557252681136
Average Independent Sensitivity: 0.8763358778625954

--------------------------------------------------

Average Cross-Validated Training Specificity: 0.8600802948387166
Average Cross-Validated Testing Specificity: 0.8403550277610712
Average Independent Specificity: 0.8424427480916034

-----------------------

In [ ]:
############FOR XGB CLASSIFIER###########

train_accuracy.clear()
test_accuracy.clear()
independent_accuracy.clear()

train_mcc.clear() #The Matthews correlation coefficient (MCC)
test_mcc.clear()
independent_mcc.clear()

train_sensitivity.clear()
test_sensitivity.clear()
independent_sensitivity.clear()

train_specificity.clear()
test_specificity.clear()
independent_specificity.clear()

j=-1
for x in (x_train_set):
    j = j+1
    #print(x.shape)
    #print(y_train.shape)
    for train_index, test_index in kf.split(x, y_train):
        subgrp_x_train = x[train_index]
        subgrp_x_test = x[test_index]

        subgrp_y_train = y_train[train_index]
        subgrp_y_test = y_train[test_index]
        #print(subgrp_x_train.shape)
        #print(subgrp_x_test.shape)

        xgb_classifier.fit(subgrp_x_train, subgrp_y_train)
        train_pred = xgb_classifier.predict(subgrp_x_train)
        test_pred = xgb_classifier.predict(subgrp_x_test)
        independent_pred = xgb_classifier.predict(x_test_set[j])


        #print(type(train_pred), type(test_pred), type(independent_pred))
        #print(train_pred.shape, test_pred.shape, independent_pred.shape)

        
        
        ###############ACCURACY#################
        train_accuracy.append(accuracy_score(subgrp_y_train, train_pred))
        test_accuracy.append(accuracy_score(subgrp_y_test, test_pred))
        independent_accuracy.append(accuracy_score(y_test, independent_pred))


        ###############MCC#################
        train_mcc.append(matthews_corrcoef(subgrp_y_train, train_pred))
        test_mcc.append(matthews_corrcoef(subgrp_y_test, test_pred))
        independent_mcc.append(matthews_corrcoef(y_test, independent_pred))


        ###############Sensitivity & Specificity#################
        tn, fp, fn, tp = confusion_matrix(subgrp_y_train, train_pred).ravel()
        train_sensitivity.append(tn/(tn+fp))
        train_specificity.append(tp/(tp+fn))

        tn, fp, fn, tp = confusion_matrix(subgrp_y_test, test_pred).ravel()
        test_sensitivity.append(tn/(tn+fp))
        test_specificity.append(tp/(tp+fn))

        tn, fp, fn, tp = confusion_matrix(y_test, independent_pred).ravel()
        independent_sensitivity.append(tn/(tn+fp))
        independent_specificity.append(tp/(tp+fn))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print("#####XGB CLASSIFIER-SUMARRY#####")
print("\n")

table_data.append('XGBC')

print("Average Cross-Validated Training Accuracy:", sum(train_accuracy)/len(train_accuracy))
table_data.append(sum(train_accuracy)/len(train_accuracy))
print("Average Cross-Validated Testing Accuracy:", sum(test_accuracy)/len(test_accuracy))
table_data.append(sum(test_accuracy)/len(test_accuracy))
print("Average Independent Accuracy:", sum(independent_accuracy)/len(independent_accuracy))
table_data.append(sum(independent_accuracy)/len(independent_accuracy))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Matthews correlation coefficient:", sum(train_mcc)/len(train_mcc))
table_data.append(sum(train_mcc)/len(train_mcc))
print("Average Cross-Validated Testing Matthews correlation coefficient:", sum(test_mcc)/len(test_mcc))
table_data.append(sum(test_mcc)/len(test_mcc))
print("Average Independent Matthews correlation coefficient:", sum(independent_mcc)/len(independent_mcc))
table_data.append(sum(independent_mcc)/len(independent_mcc))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Sensitivity:", sum(train_sensitivity)/len(train_sensitivity))
table_data.append(sum(train_sensitivity)/len(train_sensitivity))
print("Average Cross-Validated Testing Sensitivity:", sum(test_sensitivity)/len(test_sensitivity))
table_data.append(sum(test_sensitivity)/len(test_sensitivity))
print("Average Independent Sensitivity:", sum(independent_sensitivity)/len(independent_sensitivity))
table_data.append(sum(independent_sensitivity)/len(independent_sensitivity))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Specificity:", sum(train_specificity)/len(train_specificity))
table_data.append(sum(train_specificity)/len(train_specificity))
print("Average Cross-Validated Testing Specificity:", sum(test_specificity)/len(test_specificity))
table_data.append(sum(test_specificity)/len(test_specificity))
print("Average Independent Specificity:", sum(independent_specificity)/len(independent_specificity))
table_data.append(sum(independent_specificity)/len(independent_specificity))
print("\n--------------------------------------------------\n")

print(table_data)

#####XGB CLASSIFIER-SUMARRY#####


Average Cross-Validated Training Accuracy: 0.9103523035230353
Average Cross-Validated Testing Accuracy: 0.8486178861788618
Average Independent Accuracy: 0.8439694656488551

--------------------------------------------------

Average Cross-Validated Training Matthews correlation coefficient: 0.8208265834209207
Average Cross-Validated Testing Matthews correlation coefficient: 0.6973511309155976
Average Independent Matthews correlation coefficient: 0.6887497035526806

--------------------------------------------------

Average Cross-Validated Training Sensitivity: 0.9175188927975821
Average Cross-Validated Testing Sensitivity: 0.8492315135614525
Average Independent Sensitivity: 0.8641221374045801

--------------------------------------------------

Average Cross-Validated Training Specificity: 0.9031708255832855
Average Cross-Validated Testing Specificity: 0.8481902025234244
Average Independent Specificity: 0.8238167938931298

---------------------------

In [ ]:
############FOR ADABOOST###########


train_accuracy.clear()
test_accuracy.clear()
independent_accuracy.clear()

train_mcc.clear() #The Matthews correlation coefficient (MCC)
test_mcc.clear()
independent_mcc.clear()

train_sensitivity.clear()
test_sensitivity.clear()
independent_sensitivity.clear()

train_specificity.clear()
test_specificity.clear()
independent_specificity.clear()


j=-1
for x in (x_train_set):
    j = j+1
    #print(x.shape)
    #print(y_train.shape)
    for train_index, test_index in kf.split(x, y_train):
        subgrp_x_train = x[train_index]
        subgrp_x_test = x[test_index]

        subgrp_y_train = y_train[train_index]
        subgrp_y_test = y_train[test_index]
        #print(subgrp_x_train.shape)
        #print(subgrp_x_test.shape)

        adaboost_classifier.fit(subgrp_x_train, subgrp_y_train)
        train_pred = adaboost_classifier.predict(subgrp_x_train)
        test_pred = adaboost_classifier.predict(subgrp_x_test)
        independent_pred = adaboost_classifier.predict(x_test_set[j])


        #print(type(train_pred), type(test_pred), type(independent_pred))
        #print(train_pred.shape, test_pred.shape, independent_pred.shape)

        
        
        ###############ACCURACY#################
        train_accuracy.append(accuracy_score(subgrp_y_train, train_pred))
        test_accuracy.append(accuracy_score(subgrp_y_test, test_pred))
        independent_accuracy.append(accuracy_score(y_test, independent_pred))


        ###############MCC#################
        train_mcc.append(matthews_corrcoef(subgrp_y_train, train_pred))
        test_mcc.append(matthews_corrcoef(subgrp_y_test, test_pred))
        independent_mcc.append(matthews_corrcoef(y_test, independent_pred))


        ###############Sensitivity & Specificity#################
        tn, fp, fn, tp = confusion_matrix(subgrp_y_train, train_pred).ravel()
        train_sensitivity.append(tn/(tn+fp))
        train_specificity.append(tp/(tp+fn))

        tn, fp, fn, tp = confusion_matrix(subgrp_y_test, test_pred).ravel()
        test_sensitivity.append(tn/(tn+fp))
        test_specificity.append(tp/(tp+fn))

        tn, fp, fn, tp = confusion_matrix(y_test, independent_pred).ravel()
        independent_sensitivity.append(tn/(tn+fp))
        independent_specificity.append(tp/(tp+fn))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [ ]:
print("#####ADABOOST CLASSIFIER-SUMARRY#####")
print("\n")

table_data.append('ADABOOST')

print("Average Cross-Validated Training Accuracy:", sum(train_accuracy)/len(train_accuracy))
table_data.append(sum(train_accuracy)/len(train_accuracy))
print("Average Cross-Validated Testing Accuracy:", sum(test_accuracy)/len(test_accuracy))
table_data.append(sum(test_accuracy)/len(test_accuracy))
print("Average Independent Accuracy:", sum(independent_accuracy)/len(independent_accuracy))
table_data.append(sum(independent_accuracy)/len(independent_accuracy))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Matthews correlation coefficient:", sum(train_mcc)/len(train_mcc))
table_data.append(sum(train_mcc)/len(train_mcc))
print("Average Cross-Validated Testing Matthews correlation coefficient:", sum(test_mcc)/len(test_mcc))
table_data.append(sum(test_mcc)/len(test_mcc))
print("Average Independent Matthews correlation coefficient:", sum(independent_mcc)/len(independent_mcc))
table_data.append(sum(independent_mcc)/len(independent_mcc))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Sensitivity:", sum(train_sensitivity)/len(train_sensitivity))
table_data.append(sum(train_sensitivity)/len(train_sensitivity))
print("Average Cross-Validated Testing Sensitivity:", sum(test_sensitivity)/len(test_sensitivity))
table_data.append(sum(test_sensitivity)/len(test_sensitivity))
print("Average Independent Sensitivity:", sum(independent_sensitivity)/len(independent_sensitivity))
table_data.append(sum(independent_sensitivity)/len(independent_sensitivity))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Specificity:", sum(train_specificity)/len(train_specificity))
table_data.append(sum(train_specificity)/len(train_specificity))
print("Average Cross-Validated Testing Specificity:", sum(test_specificity)/len(test_specificity))
table_data.append(sum(test_specificity)/len(test_specificity))
print("Average Independent Specificity:", sum(independent_specificity)/len(independent_specificity))
table_data.append(sum(independent_specificity)/len(independent_specificity))
print("\n--------------------------------------------------\n")

print(table_data)

#####ADABOOST CLASSIFIER-SUMARRY#####


Average Cross-Validated Training Accuracy: 0.8627642276422764
Average Cross-Validated Testing Accuracy: 0.8336043360433604
Average Independent Accuracy: 0.8329770992366413

--------------------------------------------------

Average Cross-Validated Training Matthews correlation coefficient: 0.7255372605507922
Average Cross-Validated Testing Matthews correlation coefficient: 0.667451404900297
Average Independent Matthews correlation coefficient: 0.6663502543360784

--------------------------------------------------

Average Cross-Validated Training Sensitivity: 0.863697954779905
Average Cross-Validated Testing Sensitivity: 0.8343940560463075
Average Independent Sensitivity: 0.8360305343511452

--------------------------------------------------

Average Cross-Validated Training Specificity: 0.8618156339772253
Average Cross-Validated Testing Specificity: 0.8329793541198373
Average Independent Specificity: 0.8299236641221376

------------------------

In [ ]:
############FOR RANDOM FOREST CLASSIFIER###########


train_accuracy.clear()
test_accuracy.clear()
independent_accuracy.clear()

train_mcc.clear() #The Matthews correlation coefficient (MCC)
test_mcc.clear()
independent_mcc.clear()

train_sensitivity.clear()
test_sensitivity.clear()
independent_sensitivity.clear()

train_specificity.clear()
test_specificity.clear()
independent_specificity.clear()


j=-1
for x in (x_train_set):
    j = j+1
    #print(x.shape)
    #print(y_train.shape)
    for train_index, test_index in kf.split(x, y_train):
        subgrp_x_train = x[train_index]
        subgrp_x_test = x[test_index]

        subgrp_y_train = y_train[train_index]
        subgrp_y_test = y_train[test_index]
        #print(subgrp_x_train.shape)
        #print(subgrp_x_test.shape)

        random_forest_classifier.fit(subgrp_x_train, subgrp_y_train)
        train_pred = random_forest_classifier.predict(subgrp_x_train)
        test_pred = random_forest_classifier.predict(subgrp_x_test)
        independent_pred = random_forest_classifier.predict(x_test_set[j])


        #print(type(train_pred), type(test_pred), type(independent_pred))
        #print(train_pred.shape, test_pred.shape, independent_pred.shape)

        
        
        ###############ACCURACY#################
        train_accuracy.append(accuracy_score(subgrp_y_train, train_pred))
        test_accuracy.append(accuracy_score(subgrp_y_test, test_pred))
        independent_accuracy.append(accuracy_score(y_test, independent_pred))


        ###############MCC#################
        train_mcc.append(matthews_corrcoef(subgrp_y_train, train_pred))
        test_mcc.append(matthews_corrcoef(subgrp_y_test, test_pred))
        independent_mcc.append(matthews_corrcoef(y_test, independent_pred))


        ###############Sensitivity & Specificity#################
        tn, fp, fn, tp = confusion_matrix(subgrp_y_train, train_pred).ravel()
        train_sensitivity.append(tn/(tn+fp))
        train_specificity.append(tp/(tp+fn))

        tn, fp, fn, tp = confusion_matrix(subgrp_y_test, test_pred).ravel()
        test_sensitivity.append(tn/(tn+fp))
        test_specificity.append(tp/(tp+fn))

        tn, fp, fn, tp = confusion_matrix(y_test, independent_pred).ravel()
        independent_sensitivity.append(tn/(tn+fp))
        independent_specificity.append(tp/(tp+fn))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DataConversionWarning: A column-vector y was passed whe

In [ ]:
print("#####RANDOM FOREST CLASSIFIER-SUMARRY#####")
print("\n")

table_data.append('RFC')

print("Average Cross-Validated Training Accuracy:", sum(train_accuracy)/len(train_accuracy))
table_data.append(sum(train_accuracy)/len(train_accuracy))
print("Average Cross-Validated Testing Accuracy:", sum(test_accuracy)/len(test_accuracy))
table_data.append(sum(test_accuracy)/len(test_accuracy))
print("Average Independent Accuracy:", sum(independent_accuracy)/len(independent_accuracy))
table_data.append(sum(independent_accuracy)/len(independent_accuracy))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Matthews correlation coefficient:", sum(train_mcc)/len(train_mcc))
table_data.append(sum(train_mcc)/len(train_mcc))
print("Average Cross-Validated Testing Matthews correlation coefficient:", sum(test_mcc)/len(test_mcc))
table_data.append(sum(test_mcc)/len(test_mcc))
print("Average Independent Matthews correlation coefficient:", sum(independent_mcc)/len(independent_mcc))
table_data.append(sum(independent_mcc)/len(independent_mcc))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Sensitivity:", sum(train_sensitivity)/len(train_sensitivity))
table_data.append(sum(train_sensitivity)/len(train_sensitivity))
print("Average Cross-Validated Testing Sensitivity:", sum(test_sensitivity)/len(test_sensitivity))
table_data.append(sum(test_sensitivity)/len(test_sensitivity))
print("Average Independent Sensitivity:", sum(independent_sensitivity)/len(independent_sensitivity))
table_data.append(sum(independent_sensitivity)/len(independent_sensitivity))
print("\n--------------------------------------------------\n")

print("Average Cross-Validated Training Specificity:", sum(train_specificity)/len(train_specificity))
table_data.append(sum(train_specificity)/len(train_specificity))
print("Average Cross-Validated Testing Specificity:", sum(test_specificity)/len(test_specificity))
table_data.append(sum(test_specificity)/len(test_specificity))
print("Average Independent Specificity:", sum(independent_specificity)/len(independent_specificity))
table_data.append(sum(independent_specificity)/len(independent_specificity))
print("\n--------------------------------------------------\n")

print(table_data)

#####RANDOM FOREST CLASSIFIER-SUMARRY#####


Average Cross-Validated Training Accuracy: 1.0
Average Cross-Validated Testing Accuracy: 0.8236856368563688
Average Independent Accuracy: 0.8122137404580152

--------------------------------------------------

Average Cross-Validated Training Matthews correlation coefficient: 1.0
Average Cross-Validated Testing Matthews correlation coefficient: 0.6481352070404117
Average Independent Matthews correlation coefficient: 0.625159415986106

--------------------------------------------------

Average Cross-Validated Training Sensitivity: 1.0
Average Cross-Validated Testing Sensitivity: 0.8279919754958671
Average Independent Sensitivity: 0.8341984732824429

--------------------------------------------------

Average Cross-Validated Training Specificity: 1.0
Average Cross-Validated Testing Specificity: 0.8201568119609886
Average Independent Specificity: 0.7902290076335877

--------------------------------------------------

['SVC', 0.8994850948509484

In [27]:
header = [' ', 'KF TRAIN ACCURACY', 'KF TEST ACCURACY', 'IND TEST ACCURACY', 'KF TRAIN MCC', 'KF TEST MCC', 'IND TEST MCC', 'KF TRAIN SENSITIVITY', 'KF TEST SENSITIVITY', 'IND TEST SENSITIVITY', 'KF TRAIN SPECIFICITY', 'KF TEST SPECIFICITY', 'IND SPECIFICITY']

data = np.reshape(table_data, (5, 13))
print(tabulate(data, headers=header, tablefmt='fancy_grid'))

╒══════════╤═════════════════════╤════════════════════╤═════════════════════╤════════════════╤═══════════════╤════════════════╤════════════════════════╤═══════════════════════╤════════════════════════╤════════════════════════╤═══════════════════════╤═══════════════════╕
│          │   KF TRAIN ACCURACY │   KF TEST ACCURACY │   IND TEST ACCURACY │   KF TRAIN MCC │   KF TEST MCC │   IND TEST MCC │   KF TRAIN SENSITIVITY │   KF TEST SENSITIVITY │   IND TEST SENSITIVITY │   KF TRAIN SPECIFICITY │   KF TEST SPECIFICITY │   IND SPECIFICITY │
╞══════════╪═════════════════════╪════════════════════╪═════════════════════╪════════════════╪═══════════════╪════════════════╪════════════════════════╪═══════════════════════╪════════════════════════╪════════════════════════╪═══════════════════════╪═══════════════════╡
│ SVC      │            0.899485 │           0.840054 │            0.842901 │       0.799063 │      0.680511 │       0.687116 │               0.904917 │              0.846426 │           